In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
invited_info = pd.read_csv('invited_info_csv.txt', sep='\t')
question_info = pd.read_csv('question_info_csv.txt', sep='\t')
user_info = pd.read_csv('user_info_csv.txt', sep='\t')

In [4]:
all_info = invited_info.merge(question_info, left_on='qid', right_on='qid', how='left')

In [5]:
all_info = all_info.merge(user_info, left_on='uid', right_on='uid', how='left')

In [6]:
label_series = all_info['label']

In [7]:
label_array = np.array(label_series)

In [8]:
feature_dataframe = all_info.loc[:, 'qwid':'utag_142']

In [9]:
feature_name = list(feature_dataframe.columns)
# feature_name

In [10]:
feature_matrix = feature_dataframe.as_matrix()
len(feature_matrix)

229440

In [11]:
validate_nolabel = pd.read_csv('./bytecup2016data/validate_nolabel.txt', sep=',')

In [12]:
valitate_set = validate_nolabel.merge(question_info, left_on='qid', right_on='qid', how='left')

In [13]:
valitate_set = valitate_set.merge(user_info, left_on='uid', right_on='uid', how='left')

In [14]:
valitate_set_matrix = valitate_set.loc[:,'qwid':'utag_142'].as_matrix()

In [15]:
valitate_set_matrix.shape

(30466, 172)

In [16]:
import xgboost as xgb

In [25]:
dtrain = xgb.DMatrix(feature_matrix, label=label_array, feature_names=feature_name)

In [26]:
dtest = xgb.DMatrix(feature_matrix, label=label_array, feature_names=feature_name)

In [28]:
param = {'max_depth':7, 'silent':1, 'objective':'reg:logistic', 'eta': 0.015}
param['nthread'] = 4
param['eval_metric'] = 'rmse'

In [31]:
num_round = 2000

In [48]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
# eva_hist = xgb.cv(params=param, dtrain=dtrain, num_boost_round=num_round, nfold=10, early_stopping_rounds=10)

In [29]:
# eva_hist.to_csv('eva_hist.txt', index=False)

In [21]:
# eva_hist_csv = pd.read_csv('eva_hist.txt')

In [30]:
# eva_hist_csv

In [26]:
bst = xgb.train( param.items(), dtrain, num_round, )

[0]	eval-mae:0.429068	train-mae:0.438142
[1]	eval-mae:0.380064	train-mae:0.395331
[2]	eval-mae:0.345534	train-mae:0.364844
[3]	eval-mae:0.32072	train-mae:0.343159
[4]	eval-mae:0.303107	train-mae:0.327667
[5]	eval-mae:0.290227	train-mae:0.31635
[6]	eval-mae:0.280948	train-mae:0.308206
[7]	eval-mae:0.274258	train-mae:0.302058
[8]	eval-mae:0.269337	train-mae:0.297609
[9]	eval-mae:0.26586	train-mae:0.294475
[10]	eval-mae:0.263149	train-mae:0.291837
[11]	eval-mae:0.26103	train-mae:0.289918
[12]	eval-mae:0.259611	train-mae:0.288153
[13]	eval-mae:0.258366	train-mae:0.286973
[14]	eval-mae:0.257408	train-mae:0.28611
[15]	eval-mae:0.256686	train-mae:0.285447
[16]	eval-mae:0.256242	train-mae:0.284578
[17]	eval-mae:0.255531	train-mae:0.28371
[18]	eval-mae:0.255103	train-mae:0.283337
[19]	eval-mae:0.254609	train-mae:0.282831
[20]	eval-mae:0.254428	train-mae:0.282537
[21]	eval-mae:0.25416	train-mae:0.282259
[22]	eval-mae:0.254053	train-mae:0.282076
[23]	eval-mae:0.253429	train-mae:0.28133
[24]	eval-

In [27]:
# bst.get_fscore()
# bst.save_model('2000iter.model')

In [38]:
# dvalidate = xgb.DMatrix(valitate_set_matrix, feature_names=feature_name)

In [39]:
# pre = bst.predict(dvalidate)
# from graphviz import *

In [28]:
# unpredict_csv = pd.read_csv('unpredict.txt', sep=',', names=['qid','uid'])
# unpredict_csv.head()

,qid,uid
0,b06674d5f2135b4f296d25677a795f80,18ef078c925908094fa5302805a71cac Unable to pre...
1,76e176fafb7b912fce20dae5808c1692,f63e5dc51d33cebcc3f3156bca7caf05 Unable to pre...
2,45b2e6cf298a44ccfc7217aef466c65d,16090798e6130d0d85ce120032673b8b Unable to pre...
3,f20d4c2bf14232f75c333b82bd18f145,bd50e31242e9819db5074411a89f0716 Unable to pre...
4,109f6578872dcb79c52b41b78cac1cb9,e0ca1e9323dab52277c1e342aabd2f2b Unable to pre...


In [29]:
# unpredict_csv['uid'] = unpredict_csv['uid'].apply(lambda item: item.split(' ')[0])

In [30]:
# unpredict_all_csv = unpredict_csv.merge(question_info, left_on='qid', right_on='qid', how='left')
# unpredict_all_csv = unpredict_all_csv.merge(user_info, left_on='uid', right_on='uid', how='left')

In [31]:
# unpredict_all_csv.head()

,qid,uid,qwid,qcid,#upvotes,#ans,#tqans,qtag_0,qtag_1,qtag_2,...,utag_133,utag_134,utag_135,utag_136,utag_137,utag_138,utag_139,utag_140,utag_141,utag_142
0,b06674d5f2135b4f296d25677a795f80,18ef078c925908094fa5302805a71cac,-0.497299,-1.659888,-0.049869,0.174273,0.284113,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,76e176fafb7b912fce20dae5808c1692,f63e5dc51d33cebcc3f3156bca7caf05,-1.397578,-1.557129,-0.176648,-0.208696,-0.292344,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,45b2e6cf298a44ccfc7217aef466c65d,16090798e6130d0d85ce120032673b8b,-1.559045,0.097731,0.286413,-0.040820,0.521477,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,f20d4c2bf14232f75c333b82bd18f145,bd50e31242e9819db5074411a89f0716,0.585007,0.838885,-0.022560,-0.135250,0.114567,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,109f6578872dcb79c52b41b78cac1cb9,e0ca1e9323dab52277c1e342aabd2f2b,0.802581,1.179705,-0.187412,-0.098527,0.182385,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# unpredict_matrix = unpredict_all_csv.loc[:,'qwid':'utag_142'].as_matrix()

In [33]:
# d_unpredict = xgb.DMatrix(unpredict_matrix, feature_names=feature_name)

In [34]:
# pre = bst.predict(d_unpredict)
# unpredict_csv['label'] = pre

In [35]:
# unpredict_csv.to_csv('unpredict_csv.csv', index=False, header=False, sep=',')

In [41]:
# validate_nolabel['label'] = pre

NameError: name 'pre' is not defined

In [ ]:
# validate_nolabel.to_csv('validate_nolabel.csv', sep=',', index=False)

In [ ]:
# % matplotlib inline
# bst.get_fscore()